In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 300)

df = pd.read_csv("all_energy_statistics.csv", low_memory=False)

df.head()

In [ ]:
df.info(memory_usage='deep')

In [ ]:
df.country_or_area = df.country_or_area.astype('category')
df.commodity_transaction = df.commodity_transaction.astype('category')
df.unit = df.unit.astype('category')
df.category = df.category.astype('category')

In [ ]:
df.info(memory_usage='deep')

## Selecting years

In [ ]:
to_remove = df[df['country_or_area'].str.endswith('(former)')]
to_remove[['year','country_or_area']].value_counts().sort_index()

In [ ]:
to_remove['year'].value_counts().sort_index()

In [ ]:
new_df = df[df.year > 1993]
new_df.head()


### Checking the 'commodity_transaction' column

In [ ]:
new_df.commodity_transaction.value_counts()

In [ ]:
new_df.commodity_transaction.str.count(" - ").value_counts()

In [ ]:
split_commodities = new_df.commodity_transaction.str.split(" - | – ",  expand=True)
split_commodities.head(20)

In [ ]:
split_commodities.columns = ["commodity","transaction_type","additional_transaction_info"]
split_commodities.head()

In [ ]:
split_commodities.commodity = split_commodities.commodity.str.lower().str.strip()
split_commodities.commodity.value_counts().sort_index()

In [ ]:
#cleaning commodities
split_commodities.commodity = split_commodities.commodity.str.lower().str.strip()

split_commodities.commodity = split_commodities.commodity.str.replace('liquefied ', 'liquified ')
split_commodities.commodity = split_commodities.commodity.str.replace('wastes', 'waste')
split_commodities.commodity = split_commodities.commodity.str.replace('- recoverable resources', '')
split_commodities.commodity = split_commodities.commodity.str.replace('hrad', 'hard')
split_commodities.commodity.value_counts().sort_index()

In [ ]:
#cleaning transaction_type#

split_commodities.transaction_type = split_commodities.transaction_type.str.lower().str.strip()

def industry(row):
    x = str(row)
    if x.endswith('industry'):
        return row.replace('industry', '').strip()
    else:
        return row

split_commodities.transaction_type = split_commodities.transaction_type.apply(industry)
split_commodities.transaction_type = split_commodities.transaction_type.str.replace('commerce', 'commercial')
split_commodities.transaction_type = split_commodities.transaction_type.str.replace('transportation', 'transport')
split_commodities.transaction_type = split_commodities.transaction_type.str.replace('non energy', 'non-energy')
split_commodities.transaction_type = split_commodities.transaction_type.str.replace('energy uses', 'energy use')
split_commodities.transaction_type = split_commodities.transaction_type.str.replace('heat and chp plants', 'chp and heat plants')
split_commodities.transaction_type = split_commodities.transaction_type.str.replace('transformatin', 'transformation')
split_commodities.transaction_type = split_commodities.transaction_type.str.replace("consumption in", "consumption by")
split_commodities.transaction_type = split_commodities.transaction_type.str.replace("own use in", "own use by")
split_commodities.transaction_type = split_commodities.transaction_type.str.replace("transformation main ", "transformation in main")
split_commodities.transaction_type = split_commodities.transaction_type.str.replace("mainactivity", "main activity")
split_commodities.transaction_type = split_commodities.transaction_type.str.replace("non-fuel mining", "non-fuel")
split_commodities.transaction_type = split_commodities.transaction_type.str.replace("min. ind.", "")
split_commodities.transaction_type = split_commodities.transaction_type.str.replace("ind.", "")
split_commodities.transaction_type = split_commodities.transaction_type.str.lower().str.strip()

split_commodities.transaction_type.value_counts().sort_index()


In [ ]:
split_commodities.additional_transaction_info = split_commodities.additional_transaction_info.str.lower().str.strip()
split_commodities.additional_transaction_info.value_counts()

In [ ]:
split_commodities.head()

### Adding new columns to the original table

In [ ]:
new_data = pd.concat([new_df, split_commodities.commodity,split_commodities.transaction_type,split_commodities.additional_transaction_info],axis=1)
new_data.tail(20)

### Creating a file

In [ ]:
new_data.to_csv('ready_data00.csv', index=False)